# Test of Reservoir Computer prediction of Lorenz '63 System

First, import necessary modules

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from scipy.stats import pearsonr
from scipy.sparse import linalg
from scipy.linalg import solve, pinv
import matplotlib.pyplot as plt

## Define necessary  functions

In [2]:
def dxdt_lorenz(x,time,r_t, sigma = 10., beta = 8/3, rho = 28.):
    # Evaluates derivative of Lorenz '63 system with a time-dependent
    # rho value. For constant rho, input the r_t_const function.
    return np.array([sigma*(- x[0] + x[1]),\
                     r_t(time)*rho*x[0] - x[1] - x[0]*x[2],\
                     x[0]*x[1]-beta*x[2]])
    
def rk4(x, time, tau, r_t, dxdt):
    # Fourth order Runge-Kutta integrator
    
    k1 = dxdt(x, time, r_t)
    k2 = dxdt(x + k1/2*tau, time + tau/2, r_t)
    k3 = dxdt(x + k2/2*tau, time + tau/2, r_t)
    k4 = dxdt(x + tau*k3, time + tau, r_t)
    
    xnext = x + 1/6*tau*(k1+2*k2+2*k3+k4)
    return xnext

def getLorenzData(data_length, r_t, dxdt_lorenz,transient_length = 1000, tau = 0.01,sample_tau = 0.05, seed = 5):
    # Obtains time series of Lorenz '63 states after some initial transient time
    sampling_rate = round(sample_tau/tau)
    np.random.seed(seed)
    x = np.random.rand(3)
    time = -transient_length*tau
    for i in range(0,transient_length):
        x = rk4(x,time,tau,r_t,dxdt_lorenz)
        time += tau
    
    data = np.zeros((3,data_length))
    data[:,0] = x
    for i in range(0,data_length-1):
        data[:,i+1] = rk4(data[:,i],time,tau,r_t,dxdt_lorenz)
        time += tau
    
    data = data[:,::sampling_rate]
    return data

def r_t_cosine(time, period = 500, max_height = 48/28):
    # Function for oscillating rho value (not used here)
    r = 1 + (max_height-1.)/2 - (max_height-1)/2*np.cos(2*np.pi/period*time)
    return r

def r_t_const(time, value = 1):
    # Function for constant rho value
    r = value
    return r

def advanceReservoir(win,A_mat,x,u,leakage):
    # Equation for advancing reservoir state. Here, we do not use a bias factor.
    x_next = leakage*x + (1-leakage)*np.tanh(A_mat.dot(x) + np.matmul(win,u))
    return x_next

def getPrediction(win, A_mat, wout, x, predict_length, leakage,timestep):
    # Obtains a prediction of length predict_length given the trained reservoir parameters
    prediction = np.zeros((wout.shape[0],predict_length))
    aug_x = np.copy(x)
    aug_x[::2] = np.power(aug_x[::2],2)
    prediction[:,:timestep] = np.matmul(wout,aug_x)
    x_feedback = x[:,-1]
    
    for pred_idx in range(0,predict_length - timestep):
        x_feedback = advanceReservoir(win, A_mat, x_feedback, prediction[:,pred_idx], leakage)
        aug_x = np.copy(x_feedback)
        aug_x[::2] = np.power(aug_x[::2],2)
        prediction[:,pred_idx + timestep] = np.matmul(wout,aug_x)
        
    return prediction

## Next, define reservoir hyperparameters and the length of and number of predictions to be made. Finally,  we obtain a training and testing data sequence from the Lorenz '63 system.

In [3]:
input_weight = 1e-2 #Input weight for reservoir
regularization = 1e-5 #Tikhonov regularization term
average_degree = 3 #Average in-degree for each reservoir node
# forget = 1
# inv_forget = 1/forget
data_seed = 30  #Seed for initializing data
timesteps = np.arange(1,21)
leakage = 0 #Leakage parameter, unused
spectral_radius = 0.9 #Spectral radius, maximum magnitude eigenvalue of reservoir adjacency matrix 


int_step = 0.005
steps = np.arange(0.005,0.06,0.005) #Time step size for Lorenz system
num_steps = len(steps)
transient_length = 1000 #Length of initial transient input to reservoir before training begins
data_length_base = int(5e4)
data_length = data_length_base*steps/int_step #Length of full data set
train_length = 5000 #Length of training data
predict_length = 20  #Length of each prediction
predict_gap_length = 100 #Length of time between subsequent predictions
num_predictions = 3000  #Total number of predictions made

approx_num_nodes = 300 #Approximate Number of nodes in the reservoir (rounded such that each input connects to the same number of nodes)
np.random.seed(data_seed)

## Randomly generate reservoit adjacency and input matrices and initialize the reservoir node states.

In [4]:
input_size = 3
num_nodes = int(np.ceil(approx_num_nodes/input_size)*input_size); #Calculate number of nodes

# Create the adjacency matrix and set the spectral radius
A_mat = sparse.random(num_nodes,num_nodes, density = average_degree/num_nodes)
eg = linalg.eigs(A_mat, k = 1, return_eigenvectors=False)
A_mat = spectral_radius/np.abs(eg[0])*A_mat
print(A_mat.data[:10])


# Set the adjacency matrix such that all inputs go to the same number of nodes
q = int(np.floor(num_nodes/(input_size)))
win = np.zeros((num_nodes,input_size))
for i in range(input_size):
    np.random.seed(i)
    ip = (-1 + 2*np.random.randn(q));
    win[i*q:(i+1)*q,i] = input_weight*ip;

[0.28104585 0.26146857 0.1655575  0.08468628 0.52045049 0.22841742
 0.14135072 0.46482451 0.37468858 0.47175555]


## After an initial transient, record reservoir states and use them to calculate the output matrix.

In [5]:
wouts = np.zeros((input_size,num_nodes,num_steps,len(timesteps)))
#Initialize reservoir state vector x and states array
x = np.zeros((num_nodes,num_steps))
states = np.zeros((num_nodes, train_length,num_steps))
train_input_sequence = np.zeros((input_size,data_length_base,num_steps))
training_nrms_error = np.zeros((num_steps, len(timesteps)))
for i in range(num_steps):
    train_input_sequence[:,:,i] = getLorenzData(round(data_length[i]),r_t_const,dxdt_lorenz,tau = int_step,sample_tau = steps[i])
    # Advance reservoir state
    for t in range(transient_length):
        x[:,i] = advanceReservoir(win,A_mat,x[:,i],train_input_sequence[:,t,i],leakage)

    #Begin recording states for training
    states[:,0,i] = x[:,i];

    for t in range(train_length-1):
        states[:,t+1,i] = advanceReservoir(win,A_mat,states[:,t,i],train_input_sequence[:,t+transient_length,i],leakage)

    x[:,i] = states[:,-1,i]

    # We augment the reservoir states by squaring some of them to break the odd
    # symmetry of the tanh function. We find experimentally that this improves
    # performance for some systems.
    aug_states = states[:,:,i]
    aug_states[::2,:] = np.power(states[::2,:,i],2)
    idenmat = regularization*sparse.identity(num_nodes)
    states_trstates = np.matmul(aug_states,np.transpose(aug_states))
    states_trstates_inv = pinv(states_trstates + idenmat)
    norm_error = np.sqrt(np.mean(train_input_sequence[:,:,i]**2))
    for k in range(len(timesteps)):
        x_pred = states[:,-timesteps[k]:,i]
        data_trstates = np.matmul(train_input_sequence[:,transient_length+timesteps[k]-1:transient_length+train_length+timesteps[k]-1,i],np.transpose(aug_states))
        wouts[:,:,i,k] = np.matmul(data_trstates,states_trstates_inv)
        training_error = (wouts[:,:,i,k] @ aug_states) - train_input_sequence[:,transient_length+timesteps[k]-1:transient_length+train_length+timesteps[k]-1,i]
        training_nrms_error[i,k] = np.sqrt(np.mean(training_error**2))/norm_error
        if k==0:
            print(data_trstates[0:2,0])
            print(training_nrms_error[i,k])

[-3129.22580583 -2521.32886021]
1.3073920919247004e-05
[-1907.64046264 -1309.19347872]
1.2402994792524847e-05
[-2152.78860193 -1680.55538394]
1.4236714310434696e-05
[-2140.44872363 -1697.6073515 ]
1.7776833656582067e-05
[-2099.65338931 -1688.15534117]
2.1694106816420068e-05
[-2366.43952081 -1956.99494011]
2.4894103370057976e-05
[-2407.62519806 -1998.83689358]
3.07201939733896e-05
[-2366.4214733 -1956.0737886]
3.8313612641556576e-05
[-2386.87159792 -1958.58302479]
4.813664906204856e-05
[-2569.63079315 -2132.27914883]
5.875681524398331e-05
[-2452.51359422 -2003.53678984]
7.473320655471951e-05


In [6]:
pred_states = np.zeros((num_nodes, num_predictions, num_steps, len(timesteps)))
pred_nrms_error = np.zeros((num_steps,len(timesteps)))
for i in range(num_steps):
    for k in range(len(timesteps)):
        pred_states[:,0,i,k] = x[:,i]
        for t in range(num_predictions - 1):
            pred_states[:,t+1,i,k] = advanceReservoir(win,A_mat,pred_states[:,t,i,k],train_input_sequence[:,t+transient_length+train_length-1,i],leakage)
        aug_pred_states = pred_states[:,:,i,k]
        aug_pred_states[::2,:] = np.power(aug_pred_states[::2,:],2)
        onestep_pred = wouts[:,:,i,k] @ aug_pred_states
        norm_error = np.sqrt(np.mean(train_input_sequence[:,:,i]**2))
        pred_start_idx = transient_length + train_length
        pred_error = onestep_pred - train_input_sequence[:,pred_start_idx + timesteps[k] - 2:pred_start_idx + num_predictions + timesteps[k] - 2,i]
        pred_nrms_error[i,k] = np.sqrt(np.mean(pred_error**2))/norm_error
                      

In [7]:
np.savetxt('Lorenz63Data/lorenz_mutltistep_trainerror.csv', pred_nrms_error, delimiter = ',')

## We evaluate the reservoir's performance over a set of prediction periods beginning right after training ends (this is not necessary, but is easiest). We then display the valid prediction times.

In [ ]:
# Initialize data matrices
error_cutoff = 1;
predictions = np.zeros((input_size,predict_length,num_predictions))
errors = np.zeros((predict_length,num_predictions))
truths = np.zeros((input_size,predict_length,num_predictions))
valid_times = np.zeros(num_predictions)


for pred in range(num_predictions):
    # Obtain the prediction
    predictions[:,:,pred] = getPrediction(win,A_mat,wout,x_pred,predict_length,leakage,timestep)
    start_pred_idx = transient_length + train_length + pred*predict_gap_length - 1
    
    # Compare prediction to true system and calculate valid prediction time given the normalized RMS error cutoff
    truth = train_input_sequence[:,start_pred_idx:start_pred_idx + predict_length]
    truths[:,:,pred] = truth
    errors[:,pred] = np.linalg.norm(predictions[:,:,pred] - truth, axis = 0)/norm_error
    for i in range(predict_length):
        if errors[i,pred] > error_cutoff:
            break
        else:
            valid_times[pred] += 1
    
    # Advance the reservoir node state to the next prediction
    states_pred = np.zeros((num_nodes,predict_gap_length+1))
    states_pred[:,0] = x
    for i in range(predict_gap_length):
        states_pred[:,i+1] = advanceReservoir(win,A_mat,states_pred[:,i],train_input_sequence[:,start_pred_idx + i],leakage)
    x_pred = states_pred[:,-timestep:]
    x = states_pred[:,-1]

## Plot an example prediction

In [ ]:
times = np.arange(0,step*predict_length,step)
pred_idx = 10
plt.plot(times, truths[0,:,pred_idx],label = 'Truth')
plt.plot(times, predictions[0,:,pred_idx],label = 'Prediction')
plt.xlabel('Time')
plt.ylabel('x(t)')
plt.show()

plt.plot(times, truths[1,:,pred_idx],label = 'Truth')
plt.plot(times, predictions[1,:,pred_idx],label = 'Prediction')
plt.xlabel('Time')
plt.ylabel('y(t)')
plt.show()

plt.plot(times, truths[2,:,pred_idx],label = 'Truth')
plt.plot(times, predictions[2,:,pred_idx],label = 'Prediction')
plt.xlabel('Time')
plt.ylabel('z(t)')
plt.show()